# Project D12
## ADL1 - Cleaning Data

##### Opening the datasets 

In [157]:
import pandas as pd
import numpy as np
positions = pd.read_table("data/out.csv",sep = ",")
autonomy = pd.read_table("data/out_autonomy.csv",sep = ",")
speed = pd.read_table("data/out_speed.csv",sep = ",")
steer = pd.read_table("data/out_steer.csv",sep = ",")
turn = pd.read_csv("data/outturn.csv",sep = ",")

##### Convert data into pandas DataFrames

In [158]:
pos_df = pd.DataFrame(positions)
auto_df = pd.DataFrame(autonomy)
speed_df = pd.DataFrame(speed)
steer_df = pd.DataFrame(steer)
turn_df = pd.DataFrame(turn)

##### Deleting the beginning of the recording

In [159]:
def deletebeginning(pos_df):
    start_x = pos_df["field.pose.position.x"][0].round()#fixing the starting values
    print(start_x)
    start_y = pos_df["field.pose.position.y"][0].round()
    print(start_y)
    move_start_x=0#counters 
    move_start_y=0
    moved_x= False#value to represent whether the car has moved
    moved_y= False
    moved_index= 0#index counter
    for i in range(len(pos_df)):
        current_x=pos_df["field.pose.position.x"][i].round()#fixing the current x and y values
        current_y=pos_df["field.pose.position.y"][i].round()
        if current_x == start_x:# if the values do not change, they must be deleted
            move_start_x += 1
        else:
            moved_x= True
        if current_y == start_y:
            move_start_y += 1
        else:
            moved_y= True
            
        if moved_x and moved_y:# if the x and y values did change
            if move_start_x > move_start_y:#finding out which value changed last
                moved_index= move_start_x
            elif move_start_x < move_start_y:
                moved_index= move_start_y
    removable_indicies = pos_df[:moved_index]
    for index in range(len(removable_indicies)):
        remove_closest_time(speed_df, pos_df, index)
        remove_closest_time(steer_df, pos_df, index)
        remove_closest_time(turn_df, pos_df, index)
        pos_df.drop(index, 0, inplace=True)
        pos_df.reset_index(drop=True, inplace=True)
    print(moved_index)
    return pos_df

In [160]:
def deleteend(pos_df):
    start_x = pos_df["field.pose.position.x"][len(pos_df["field.pose.position.x"])-1].round()
    start_y = pos_df["field.pose.position.y"][len(pos_df["field.pose.position.y"])-1].round()
    stop_start_x=0
    stop_start_y=0
    stopped_x= False
    stopped_y= False
    stopped_index= 0
    print(start_x)
    print(start_y)
    i = len(pos_df["field.pose.position.x"])-1
    while i > 0:
        current_x=pos_df["field.pose.position.x"][i].round()
        current_y=pos_df["field.pose.position.y"][i].round()
        if current_x == start_x:
            stop_start_x += 1
        else:
            stopped_x= True
        if current_y == start_y:
            stop_start_y += 1
        else:
            stopped_y= True
            
        if stopped_x and stopped_y:
            if stop_start_x > stop_start_y:
                stopped_index= stop_start_x
            elif stop_start_x < stop_start_y:
                stopped_index= stop_start_y
        i -= 1
    
    real_stopped_index = len(pos_df["field.pose.position.x"])-stopped_index
    removable_indicies = pos_df[real_stopped_index:]
    for index in range(len(pos_df["field.pose.position.x"])-1, real_stopped_index-1, -1):
        remove_closest_time(speed_df, pos_df, index)
        remove_closest_time(steer_df, pos_df, index)
        remove_closest_time(turn_df, pos_df, index)
        pos_df.drop(index, 0, inplace=True)
        pos_df.reset_index(drop=True, inplace=True)
        
    print(real_stopped_index)
    return pos_df

In [161]:
pos_df = deletebeginning(pos_df)

-8354.0
-5678.0
111


In [162]:
pos_df = deleteend(pos_df)

-11672.0
-3078.0
54772


In [163]:
pos_df

,%time,field.header.seq,field.header.stamp,field.header.frame_id,field.pose.position.x,field.pose.position.y,field.pose.position.z,field.pose.orientation.x,field.pose.orientation.y,field.pose.orientation.z,field.pose.orientation.w
0,1635234546782376676,13641,1635234546781745556,map,-8354.300180,-5678.253026,65.867257,0.003351,-0.011833,0.833959,0.551689
1,1635234546822177181,13643,1635234546821535288,map,-8354.300188,-5678.252989,65.867358,0.003342,-0.011829,0.833960,0.551688
2,1635234546862063408,13645,1635234546861629707,map,-8354.300203,-5678.252961,65.867371,0.003336,-0.011836,0.833967,0.551678
3,1635234546903314414,13647,1635234546902683283,map,-8354.300230,-5678.252952,65.867383,0.003348,-0.011844,0.833964,0.551682
4,1635234546941999948,13649,1635234546941526382,map,-8354.300264,-5678.252964,65.867426,0.003344,-0.011842,0.833965,0.551680
...,...,...,...,...,...,...,...,...,...,...,...
54767,1635235644323060507,68518,1635235644321515155,map,-11670.575787,-3077.307902,82.851293,-0.013078,0.011809,0.977179,-0.211683
54768,1635235644342601887,68519,1635235644341446104,map,-11670.622551,-3077.329696,82.850943,-0.012979,0.011713,0.977196,-0.211618
54769,1635235644362599789,68520,1635235644361646792,map,-11670.668480,-3077.351145,82.850606,-0.012841,0.011554,0.977213,-0.211556
54770,1635235644382919547,68521,1635235644381533183,map,-11670.713768,-3077.372231,82.850281,-0.012706,0.011530,0.977230,-0.211488


In [164]:
speed_df

,%time,field.header.seq,field.header.stamp,field.header.frame_id,field.vehicle_speed,field.vehicle_speed_valid
0,1635234546864484937,355,1635234546864319883,pacmod,0.00,1
1,1635234547098776438,362,1635234547098631913,pacmod,0.00,1
2,1635234547303861140,368,1635234547300741640,pacmod,0.00,1
3,1635234547503227658,374,1635234547501411829,pacmod,0.00,1
4,1635234547702713230,380,1635234547702407433,pacmod,0.00,1
...,...,...,...,...,...,...
32663,1635235644243438360,33126,1635235644243162430,pacmod,2.83,1
32664,1635235644276275451,33127,1635235644275973024,pacmod,2.73,1
32665,1635235644310914180,33128,1635235644310496777,pacmod,2.65,1
32666,1635235644342983498,33129,1635235644342687054,pacmod,2.60,1


In [165]:
steer_df

,%time,field.header.seq,field.header.stamp,field.header.frame_id,field.enabled,field.override_active,field.command_output_fault,field.input_output_fault,field.output_reported_fault,field.pacmod_fault,field.vehicle_fault,field.manual_input,field.command,field.output
0,1635234546808951031,353,1635234546807354837,pacmod,0,0,0,0,0,0,0,-0.333,0.0,-0.333
1,1635234546860799631,355,1635234546860574907,pacmod,0,0,0,0,0,0,0,-0.333,0.0,-0.333
2,1635234547062727348,361,1635234547061918220,pacmod,0,0,0,0,0,0,0,-0.285,0.0,-0.285
3,1635234547263067290,367,1635234547262769576,pacmod,0,0,0,0,0,0,0,-0.091,0.0,-0.091
4,1635234547464002357,373,1635234547463748894,pacmod,0,0,0,0,0,0,0,0.020,0.0,0.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32664,1635235644240526365,33126,1635235644240201419,pacmod,0,0,0,0,0,0,0,-0.004,0.0,-0.004
32665,1635235644272793332,33127,1635235644272432385,pacmod,0,0,0,0,0,0,0,-0.004,0.0,-0.004
32666,1635235644307685490,33128,1635235644307202780,pacmod,0,0,0,0,0,0,0,-0.004,0.0,-0.004
32667,1635235644339845044,33129,1635235644339506566,pacmod,0,0,0,0,0,0,0,-0.004,0.0,-0.004


In [166]:
turn_df

,%time,field.header.seq,field.header.stamp,field.header.frame_id,field.enabled,field.override_active,field.command_output_fault,field.input_output_fault,field.output_reported_fault,field.pacmod_fault,field.vehicle_fault,field.manual_input,field.command,field.output
0,1635234546862016814,355,1635234546861975939,pacmod,0,0,0,0,0,0,0,1,1,1
1,1635234547063472394,361,1635234547062803212,pacmod,0,0,0,0,0,0,0,1,1,1
2,1635234547263898695,367,1635234547263683594,pacmod,0,0,0,0,0,0,0,1,1,1
3,1635234547500228288,374,1635234547499478558,pacmod,0,0,0,0,0,0,0,1,1,1
4,1635234547700716229,380,1635234547700384030,pacmod,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32663,1635235644241527288,33126,1635235644241120709,pacmod,0,0,0,0,0,0,0,1,1,1
32664,1635235644273621505,33127,1635235644273404553,pacmod,0,0,0,0,0,0,0,1,1,1
32665,1635235644308922881,33128,1635235644308207434,pacmod,0,0,0,0,0,0,0,1,1,1
32666,1635235644340890075,33129,1635235644340682089,pacmod,0,0,0,0,0,0,0,1,1,1


In [167]:
#pos_df: 20000003.229291935
#speed_df: 33486280.14943612
#steer_df: 33485068.742158055
#turn_df: 33486269.028086815
results = [20000003.229291935, 33486280.14943612, 33485068.742158055, 33486269.028086815]
print(min(results))

20000003.229291935


In [168]:
def remove_closest_time(remove_table, origin_table, removed_index):
    #remove_table is the table from where we remove the row
    #origin_table is the table, where the original timestamp is
    #removed_index is the index in the origin table which timestamp we wish to compare
    closest = 10000000000000
    index = 0
    best_index = 0
    freq = 20000003.229291935 # best frequency across the tables from pos_df
    origin_table_time = origin_table["field.header.stamp"][removed_index] # time we wish to compare to other tables
    for time in remove_table["field.header.stamp"]:
        if abs(origin_table_time - time) < closest: # finding new best time compared to original
            closest = abs(origin_table_time - time)
            best_index = index
        index += 1
    if (closest / freq) < 1: # check to see whether the removable time seems valid or is unrelated.
        remove_table.drop(best_index, 0, inplace=True) # removes row
        remove_table.reset_index(drop=True, inplace=True) # resets indecies

In [170]:
def deleteturnsignalon():
    timestamptobedeleted = []
    for i in range(len(turn_df)):
        if turn_df["field.enabled"][i] == 1:
            timestamptobedeleted.append(i)
        for index in timestamptobedeleted:
            remove_closest_time(pos_df,turn_df,index)
            remove_closest_time(speed_df,turn_df,index)
            remove_closest_time(steer_df,turn_df,index)
            turn_df.drop(index, 0, inplace=True)
            turn_df.reset_index(drop=True, inplace=True)

In [171]:
deleteturnsignalon()
turn_df

,%time,field.header.seq,field.header.stamp,field.header.frame_id,field.enabled,field.override_active,field.command_output_fault,field.input_output_fault,field.output_reported_fault,field.pacmod_fault,field.vehicle_fault,field.manual_input,field.command,field.output
0,1635234546862016814,355,1635234546861975939,pacmod,0,0,0,0,0,0,0,1,1,1
1,1635234547063472394,361,1635234547062803212,pacmod,0,0,0,0,0,0,0,1,1,1
2,1635234547263898695,367,1635234547263683594,pacmod,0,0,0,0,0,0,0,1,1,1
3,1635234547500228288,374,1635234547499478558,pacmod,0,0,0,0,0,0,0,1,1,1
4,1635234547700716229,380,1635234547700384030,pacmod,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32663,1635235644241527288,33126,1635235644241120709,pacmod,0,0,0,0,0,0,0,1,1,1
32664,1635235644273621505,33127,1635235644273404553,pacmod,0,0,0,0,0,0,0,1,1,1
32665,1635235644308922881,33128,1635235644308207434,pacmod,0,0,0,0,0,0,0,1,1,1
32666,1635235644340890075,33129,1635235644340682089,pacmod,0,0,0,0,0,0,0,1,1,1


In [ ]:
def separatecurves():
    turnvalue = 1
    curvelist = [][]
    curvecounter = 0
    curvepart = 0
    for i in range(len(pos_df)):
        for j in range(len(turn_df)):
            if pos_df["field.header.stamp"][i] < turn_df["field.header.stamp"][j] or pos_df["field.header.stamp"][i] == turn_df["field.header.stamp"][j]:
                turnvalue = turn_df["field.output"][j]
                if turn_df["field.output"][j] == turnvalue and turnvalue != 1:
                    curvelist[curvecounter][curvepart]= i
                    i += 1
                    curvepart += 1
                    
        curvecounter += 1
                        
                       